<a href="https://colab.research.google.com/github/karthikk9900/NLP/blob/main/nlp_covid_fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix, accuracy_score
import pandas as pd
import numpy as np
import pickle
from sklearn.linear_model import LogisticRegressionCV
import re
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df=pd.read_csv('covid_fake.csv')
df.head()

,title,text,source,label
0,Due to the recent outbreak for the Coronavirus...,"You just need to add water, and the drugs and ...",coronavirusmedicalkit.com,Fake
1,NaN,Hydroxychloroquine has been shown to have a 10...,RudyGiuliani,Fake
2,NaN,Fact: Hydroxychloroquine has been shown to hav...,CharlieKirk,Fake
3,NaN,The Corona virus is a man made virus created i...,JoanneWrightForCongress,Fake
4,NaN,Doesn’t @BillGates finance research at the Wuh...,JoanneWrightForCongress,Fake


In [ ]:
df.shape

(1164, 4)

In [ ]:
df['label'].value_counts()

,count
label,
TRUE,584
Fake,345
fake,230


In [ ]:
df.loc[5:15]

,title,text,source,label
5,CORONA UNMASKED: Chinese Intelligence Officer ...,NaN,NaN,NaN
6,NaN,Urgent: Health Bulletin to the Public. Ministr...,Ministry of Health,Fake
7,NaN,"Pls tell ur families, relatives and friendsMOH...",NWLLAB,Fake
8,NaN,SERIOUS EXCELLENT ADVICE by Japanese doctors t...,Japanese doctors treating COVID-19 cases,Fake
9,Basic protective measures against the new coro...,Stay aware of the latest information on the CO...,https://www.who.int/emergencies/diseases/novel...,TRUE
10,NaN,The new Coronavirus may not show signs of infe...,Taiwan Experts,Fake
11,NaN,A vaccine meant for cattle can be used to figh...,facebook,Fake
12,NaN,Using a hair dryer to breathe in hot air can c...,Youtube,Fake
13,NaN,Corona virus before it reaches the lungs it re...,twitter,Fake
14,Exposing yourself to the sun or to temperature...,"You can catch COVID-19, no matter how sunny or...",https://www.who.int/emergencies/diseases/novel...,TRUE


In [ ]:
df.isna().sum()

,0
title,82
text,10
source,20
label,5


In [ ]:
df.loc[df['label']=='FAKE',['label']]='FAKE'
df.loc[df['label']=='fake',['label']]='FAKE'
df.loc[df['source']=='facebook',['source']]='Facebook'
df.text.fillna(df.title,inplace=True)
df.loc[5]['label']='FAKE'
df.loc[15]['label']='TRUE'
df.loc[43]['label']='FAKE'
df.loc[131]['label']='TRUE'
df.loc[242]['label']='FAKE'
df.title.fillna('missing',inplace=True)
df.source.fillna('missing',inplace=True)
df['title_text']=df['title']+' '+df['text']

In [ ]:
df.head()

,title,text,source,label,title_text
0,Due to the recent outbreak for the Coronavirus...,"You just need to add water, and the drugs and ...",coronavirusmedicalkit.com,Fake,Due to the recent outbreak for the Coronavirus...
1,missing,Hydroxychloroquine has been shown to have a 10...,RudyGiuliani,Fake,missing Hydroxychloroquine has been shown to h...
2,missing,Fact: Hydroxychloroquine has been shown to hav...,CharlieKirk,Fake,missing Fact: Hydroxychloroquine has been show...
3,missing,The Corona virus is a man made virus created i...,JoanneWrightForCongress,Fake,missing The Corona virus is a man made virus c...
4,missing,Doesn’t @BillGates finance research at the Wuh...,JoanneWrightForCongress,Fake,missing Doesn’t @BillGates finance research at...


In [ ]:
df.shape

(1164, 5)

In [ ]:
df['title_text'][101]

'Yes, coronavirus is a BIOWEAPON with gene sequencing that’s only possible if it was genetically modified in a lab The truth about novel coronavirus is starting to trickle its way out of the realm of independent science with new research pointing to strange anomalies in the virus’s genetic structure that suggest it’s more than likely a bioweapon. Published in the online journal bioRxiv, the study found that novel coronavirus contains “key structural proteins” from HIV. Entitled, “Uncanny similarity of unique inserts in the 2019-nCoV spike protein to HIV-1 gp120 and Gag,” the paper identifies four unique insertions in the virus’s spike glycoprotein that aren’t present in any other form of coronavirus, meaning this one is a whole different animal. The paper goes into further specifics about how these inserts do not appear to be natural, concluding that the engineering of novel coronavirus with these unusual gene sequences “is unlikely to be fortuitous in nature,” again pointing to its un

In [ ]:
def preprocessor(text):
  text=re.sub('<[^>]*>','',text)
  text=re.sub(r'[^\w\s]','',text)
  text=re.sub(r'[\n]','',text)
  text=text.lower()
  return text
df['title_text']=df['title_text'].apply(preprocessor)
df['title_text'][101]

'yes coronavirus is a bioweapon with gene sequencing thats only possible if it was genetically modified in a lab the truth about novel coronavirus is starting to trickle its way out of the realm of independent science with new research pointing to strange anomalies in the viruss genetic structure that suggest its more than likely a bioweapon published in the online journal biorxiv the study found that novel coronavirus contains key structural proteins from hiv entitled uncanny similarity of unique inserts in the 2019ncov spike protein to hiv1 gp120 and gag the paper identifies four unique insertions in the viruss spike glycoprotein that arent present in any other form of coronavirus meaning this one is a whole different animal the paper goes into further specifics about how these inserts do not appear to be natural concluding that the engineering of novel coronavirus with these unusual gene sequences is unlikely to be fortuitous in nature again pointing to its unnatural origin almost n

In [ ]:
porter =PorterStemmer()
def tokenizer_porter(text):
  return [porter.stem(word) for word in text.split()]

In [ ]:

tfidf=TfidfVectorizer(strip_accents=None,lowercase=False,preprocessor=None,tokenizer=tokenizer_porter,use_idf=True,norm='l2',smooth_idf=True)
x=tfidf.fit_transform(df['title_text'])
y=df.label.values


In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=0,test_size=0.3,shuffle=True)

In [ ]:
clf=LogisticRegressionCV(cv=5,scoring='accuracy',random_state=0,n_jobs=-1,verbose=3,max_iter=300)
clf.fit(x_train,y_train)

fake_news_model=open('fake_news_model.pkl','wb')
pickle.dump(clf,fake_news_model)
fake_news_model.close()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   52.3s finished


In [ ]:
filename='fake_news_model.pkl'
saved_clf=pickle.load(open(filename,'rb'))

saved_clf.score(x_test,y_test)

0.7828571428571428

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report # Import classification_report
y_pred=saved_clf.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

        FAKE       0.73      0.34      0.47        79
        Fake       0.61      0.83      0.70        99
        TRUE       0.92      0.96      0.94       172

    accuracy                           0.78       350
   macro avg       0.75      0.71      0.70       350
weighted avg       0.79      0.78      0.77       350

